# Noisy circuits with Pennylane

This notebook will follow the demostration given by Xanadu por using the Quantum Machine Learning tool Pennylane for simulate noisy quantum circuits which is available [here](https://pennylane.ai/qml/demos/tutorial_noisy_circuits.html)

-------------

Noise can be defined as any unwanted operation that is done over or circuit, this can be separated into **coherent noise** and **incoherent noise**.

The coherent noise ocours when we don't apply the exact transformation we were expecting to apply. This means that the physical way in which we are doing the transformations is not absolutely exact. This errors commonly come from imperfect calibrated devices, so when we are trying to apply a gate for example a rotation of angle $\phi$ this is really applying a rotarion of $\phi + \epsilon$.

The incoherent noise is more problematic, because it came from interaction of our qubit with the enviorement. It is difficult to determine how this kind of  perturbations is going to change our system, so then this results in random rotarions of our qubits. an example can be given when we initialize a qubit in the $|0\rangle$ state, if you do this several times you will find that approximately $1\%$ of the times that qubit results to be flipped into the state $|1\rangle$.

Now we have de density matrix, this is a mathematical object which describes the quantum state of a system. This operator lets us to easily distinguish if a system is either in a pure or mixed state by calculating $Tr(\rho^2)$. If $Tr(\rho^2) = 1$ the system is in a pure state, if $Tr(\rho^2)<1$ it is in a mixed one.

Here we will learn a little bit about PennyLane's *default.mixed*. This is a device that provides support for mixed states and for simulating noisy computations. Let's simulate a Bell state $|\psi\rangle = \frac{1}{\sqrt{2}}\left(|00\rangle + |11\rangle\right)$. We ask the QNode to return the expectation value of $Z_0 \otimes Z_1$

In [1]:
# Let's import PennyLane's library
import pennylane as qml
from pennylane import numpy as np

In [3]:
# I understand this is an initialization of a 2 qubit circuit
dev = qml.device('default.mixed',wires=2)

@qml.qnode(dev)

def circuit():
    
    qml.Hadamard(wires=0)
    qml.CNOT(wires=[0,1])
    return qml.expval(qml.PauliZ(0)@qml.PauliZ(1))

print(f"Qnode output={circuit():.4f}")

Qnode output=1.0000


This decive stores the quantum state as a density matrix. In this case is the matrix associated to the Bell state.

In [4]:
print(f"Output state matrix is = \n{np.real(dev.state)}")

Output state matrix is = 
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]


Quantum chanels can be mathematically described as linear, completely positive and trace-preserving (CPTP) maps. This channels are used for noise modeling. A way to describe this channels is through Kraus operators $\{K_i\}$. This operators satisfy the condition $$\sum_i K_i^\dagger K_i = I$$.
Given an initial state $\rho$, the result after the action of a $\Phi$ channel is
$$\Phi(\rho) = \sum_i K_i\rho K_i^\dagger$$

**Just like pure states are special cases of mixed states, unitary transformations are spacial cases of quantum channels**. Then the general description for the evolution of a quantum state should be given in terms of quantum channeld, which could take into account perturbations over the system, it means, energy loss due to interaction with the enviroment.
Then unitary transformations  are represented by a **single** Kraus operator, then it is immediate that an unitary operator $U$ acts over a system transforming it into $U\rho U^\dagger$.